In [ ]:
import geopandas as gpd
import pandas as pd

In [ ]:
name_aree = "AreeIdonee_CLC18"
AreeIdonee = gpd.read_file("..\\" + name_aree + ".gpkg")

In [ ]:
AreeIdoneeNoVincoli = pd.DataFrame([])
removed = []
fixed_AI = []
fixed_CM = []
wrong = []
remaining = []

In [ ]:
field = "Code_18"
glacers = "335"

# field = "lcp"
# AreeIdonee[field] = AreeIdonee[field].astype(str)
# glacers = "33500"

woods = "31"
water = "5"
agri = "2"


In [ ]:
for index, row in AreeIdonee.iterrows():
    if "AI" in row.TipologiaArea and row[field].startswith(agri):
        AreeIdoneeNoVincoli = pd.concat([AreeIdoneeNoVincoli,pd.DataFrame(row)], axis=1)
        remaining.append(str(row[field]) + " " + str(row['TipologiaArea']))
    elif "CM" in row.TipologiaArea and row[field].startswith(agri):
        AreeIdoneeNoVincoli = pd.concat([AreeIdoneeNoVincoli,pd.DataFrame(row)], axis=1)
        remaining.append(str(row[field]) + " " + str(row['TipologiaArea']))
    elif row.TipologiaArea != "AI" and row.TipologiaArea != "CM" and not str(row[field]) == glacers and not str(row[field]).startswith(woods) and not str(row[field]).startswith(water):
        if "AI" in row.TipologiaArea and "CM" in row.TipologiaArea:
            row.TipologiaArea = row.TipologiaArea.replace("AI","")
            row.TipologiaArea = row.TipologiaArea.replace("CM","")
            if row.TipologiaArea == "":
                wrong.append(str(row[field]) + " " + str(row['TipologiaArea']))
            else:
                AreeIdoneeNoVincoli = pd.concat([AreeIdoneeNoVincoli,pd.DataFrame(row)], axis=1)
                fixed_AI.append(str(row[field]) + " " + str(row['TipologiaArea']))
        elif "AI" in row.TipologiaArea:
            row.TipologiaArea = row.TipologiaArea.replace("AI","")
            AreeIdoneeNoVincoli = pd.concat([AreeIdoneeNoVincoli,pd.DataFrame(row)], axis=1)
            fixed_AI.append(str(row[field]) + " " + str(row['TipologiaArea']))
        elif "CM" in row.TipologiaArea:
            row.TipologiaArea = row.TipologiaArea.replace("CM","")
            AreeIdoneeNoVincoli = pd.concat([AreeIdoneeNoVincoli,pd.DataFrame(row)], axis=1)
            fixed_CM.append(str(row[field]) + " " + str(row['TipologiaArea']))
        else:
            AreeIdoneeNoVincoli = pd.concat([AreeIdoneeNoVincoli,pd.DataFrame(row)], axis=1)
            remaining.append(str(row[field]) + " " + str(row['TipologiaArea']))
    else:
        removed.append(str(row[field]) + " " + str(row['TipologiaArea']))

In [ ]:
AreeIdoneeNoVincoli_df = AreeIdoneeNoVincoli.T

AreeIdoneeNoVincoli_df["AT"] = AreeIdoneeNoVincoli_df.TipologiaArea.apply(lambda x: "si" if 'AT' in str(x) else "no")
AreeIdoneeNoVincoli_df["AI"] = AreeIdoneeNoVincoli_df.TipologiaArea.apply(lambda x: "si" if 'AI' in str(x) else "no")
AreeIdoneeNoVincoli_df["RA"] = AreeIdoneeNoVincoli_df.TipologiaArea.apply(lambda x: "si" if 'RA' in str(x) else "no")
AreeIdoneeNoVincoli_df["CM"] = AreeIdoneeNoVincoli_df.TipologiaArea.apply(lambda x: "si" if 'CM' in str(x) else "no")

In [ ]:
AreeIdoneeNoVincoli_geo = gpd.GeoDataFrame(AreeIdoneeNoVincoli_df, geometry=AreeIdoneeNoVincoli_df.geometry)
AreeIdoneeNoVincoli_geo.crs = AreeIdonee.crs
AreeIdoneeNoVincoli_geo.to_file("..\\" + name_aree + "_fix.gpkg", driver='GPKG')